In [2]:
!pip install chars2vec

%tensorflow_version 1.x
import tensorflow as tf
import json
import pickle
import numpy as np
import chars2vec
import random

     |████████████████████████████████| 8.1MB 5.5MB/s 
  Created wheel for chars2vec: filename=chars2vec-0.1.7-cp36-none-any.whl size=8111096 sha256=aa3570d06d190791865be16d18cff1381b8eb780d56c54140e8a0cd29b1d86cd
  Stored in directory: /root/.cache/pip/wheels/97/b6/65/d7e778ef1213ec77d315aea0f536068b96e36cc94c02abbfde
Successfully built chars2vec
TensorFlow 1.x selected.


Using TensorFlow backend.


In [3]:
with open('data/2k_references',encoding='UTF-8') as file:
  filedata=json.load(file)

In [4]:
def create_styles(strng):
    strng = ''.join([i for i in strng if not i.isdigit()])
    str1=strng.strip()
    str_split=str1.split(" ")
    names=[]
    if len(str_split)==1:
        names.append(str_split)
        
    elif len(str_split)==2:
        style1= str1
        style2=str_split[0][0] + " " + str_split[-1] 
        style3= str_split[-1][0]+ " "+ str_split[0]
        name_list=(style1, style2,style3)
        names.append(name_list)
    
    else:
        style1= str1
        style2=str_split[0][0]+" "+str_split[1][0]+" " + str_split[-1] 
        style3= str_split[-1]+" " +str_split[0][0]+""
        style4= str_split[-1]+ " " +str_split[0][0] + " " + str_split[1]
        name_list=(style1,style2,style3,style4)
        names.append(name_list)
    
    return names

In [5]:
unique_auth = set()
count=0
for i in filedata:
  unique_auth.add(i['Author'])
  for j in i['coauthor']:
      unique_auth.add(j)
        
auth_styles=[]
for auth in unique_auth:
    for i in create_styles(auth):
        auth_styles.append(tuple(i))

In [8]:
# Generate positive samples
pos_sam=[]
count=0
for styl in auth_styles:
    if len(styl)==1:
      continue
    if len(styl)==3:
        pos_sam.append(tuple(styl[0:2]))
        pos_sam.append(tuple([styl[0]]+[styl[2]]))
    else:
        pos_sam.append(styl[0:2])
        pos_sam.append(tuple([styl[0]]+[styl[2]]))
        pos_sam.append(tuple([styl[0]]+[styl[3]]))
pos_sam

# Generate negative samples
auth_index={}
i=0
for author in unique_auth:
    auth=''.join([i for i in author if not i.isdigit()])
    auth_index[i]=auth
    i+=1

neg_sam=[]
end=len(unique_auth) - 1
sample_len=len(pos_sam)
for i in range(sample_len):
    n1 = random.randint(0,end)
    r = list(range(1,n1)) + list(range(n1+1, end))
    n2 = random.choice(r)
    neg_sam.append(tuple([auth_index[n1],auth_index[n2]]))

pos_y=np.zeros(len(pos_sam),dtype='int')
neg_y=np.ones(len(neg_sam),dtype='int')

x_train=[]
for i in pos_sam:
    x_train.append(tuple(i))
for i in neg_sam:
    x_train.append(tuple(i))
y_train=np.append(pos_y,neg_y)


In [10]:
model_chars = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.',
               '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<',
               '=', '>', '?', '@', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
               'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w',
               'x', 'y', 'z']

model_ch=list(set(''.join(i for i in unique_auth).lower())) 
 
model_ch=list(set(np.append(model_chars,model_ch)))

epochs=20

path_to_model = 'data/Auth2Vec_model'
dim=200
c2v_model = chars2vec.train_model(dim, x_train, y_train, model_ch,max_epochs=epochs)
chars2vec.save_model(c2v_model, path_to_model)
c2v_model = chars2vec.load_model(path_to_model)


Train on 6175 samples, validate on 325 samples
Epoch 1/20
6175/6175 [==============================] - 54s 9ms/step - loss: 0.4514 - val_loss: 0.2001
Epoch 2/20
6175/6175 [==============================] - 52s 8ms/step - loss: 0.3791 - val_loss: 0.1484
Epoch 3/20
6175/6175 [==============================] - 52s 8ms/step - loss: 0.3541 - val_loss: 0.1316
Epoch 4/20
6175/6175 [==============================] - 58s 9ms/step - loss: 0.3346 - val_loss: 0.1498
Epoch 5/20
6175/6175 [==============================] - 53s 9ms/step - loss: 0.3197 - val_loss: 0.1327
Epoch 6/20
6175/6175 [==============================] - 54s 9ms/step - loss: 0.3054 - val_loss: 0.1445
Epoch 7/20
6175/6175 [==============================] - 53s 9ms/step - loss: 0.2911 - val_loss: 0.1295
Epoch 8/20
6175/6175 [==============================] - 53s 9ms/step - loss: 0.2785 - val_loss: 0.1497
Epoch 9/20
6175/6175 [==============================] - 53s 9ms/step - loss: 0.2628 - val_loss: 0.1556
Epoch 10/20
6175/6175 [===